In [0]:
#comment


In [0]:
unemployment = spark.read.format("csv").option('header', 'True').load("abfss://landing-zone@20231113desa.dfs.core.windows.net/usa_spending/external_data/Unemployment.csv")

In [0]:
display(unemployment)

In [0]:
unemployment = unemployment.select('FIPS_Code', 'unemployment_rate_2019', 'unemployment_rate_2020', 'unemployment_rate_2021', 'unemployment_rate_2022', 'median_household_income_2021')

In [0]:
unemployment.display()

In [0]:
congressional = spark.read.format("csv").option('header', 'true').load("abfss://landing-zone@20231113desa.dfs.core.windows.net/usa_spending/external_data/1976-2022-house.csv")

In [0]:
congressional.display()

In [0]:
from pyspark.sql.functions import regexp_replace

congressional = congressional.withColumn('district', regexp_replace('district', r'^[0]', ''))

In [0]:
congressional.display()

In [0]:
congressional.display()

In [0]:
congressional = congressional.where("year >= 2018")

In [0]:
congressional.display()

In [0]:
congressional = congressional.select('year', 'state', 'state_fips', 'district', 'party', 'candidatevotes')

In [0]:
congressional.display()

In [0]:
from pyspark.sql.types import IntegerType
congressional = congressional.withColumn('candidatevotes', congressional.candidatevotes.cast(IntegerType()))

In [0]:
winner_df = congressional.groupBy('year', 'state', 'state_fips', 'district').max('candidatevotes')

In [0]:
final_df = winner_df.join(congressional, ['state_fips', 'district', 'year', 'state'], how='inner')
display(final_df)

In [0]:
final_df = final_df.drop('candidatevotes')

In [0]:
final_df.display()